In [20]:
import pandas as pd
customer_master = pd.read_csv('1/customer_master.csv')
item_master = pd.read_csv('1/item_master.csv')
transaction_1 = pd.read_csv('1/transaction_1.csv')
transaction_2 = pd.read_csv('1/transaction_2.csv')
transaction = pd.concat([transaction_1, transaction_2], ignore_index=True)
transaction_detail_1 = pd.read_csv('1/transaction_detail_1.csv')
transaction_detail_2 = pd.read_csv('1/transaction_detail_2.csv')
transaction_details = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index=True)

join_data = pd.merge(transaction_details, transaction[['transaction_id', 'payment_date', 'customer_id']], on='transaction_id', how='left')
join_data = pd.merge(join_data, customer_master, on='customer_id', how='left')
join_data = pd.merge(join_data, item_master, on='item_id', how='left')
join_data.head()





,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,customer_name_kana,email,gender,age,birth,pref,item_name,item_price
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,井本 芳正,2019-01-07 14:34:35,いもと よしまさ,imoto_yoshimasa@example.com,M,30,1989/7/15,熊本県,PC-E,210000
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019,三船 六郎,2019-01-27 18:00:11,みふね ろくろう,mifune_rokurou@example.com,M,73,1945/11/29,京都府,PC-A,50000
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120,山根 小雁,2019-01-11 08:16:02,やまね こがん,yamane_kogan@example.com,M,42,1977/5/17,茨城県,PC-C,120000
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215,池田 菜摘,2019-01-10 05:07:38,いけだ なつみ,ikeda_natsumi@example.com,F,47,1972/3/17,兵庫県,PC-E,210000
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865,栗田 憲一,2019-01-25 06:46:05,くりた けんいち,kurita_kenichi@example.com,M,74,1944/12/17,長崎県,PC-B,85000
